Pruebesita de celepe
safkjdsnjvbsdv dsnm

In [1]:
print("hola.")

hola.


In [5]:
import pandas as pd
import numpy as np



In [9]:
Data = pd.read_csv('DATA.csv')
print(Data)
Data.describe()

     STUDENT ID   1   2   3   4   5   6   7   8   9   ...  23   24   25   26   \
0    STUDENT1      2   2   3   3   1   2   2   1   1  ...    1    1    3    2   
1    STUDENT2      2   2   3   3   1   2   2   1   1  ...    1    1    3    2   
2    STUDENT3      2   2   2   3   2   2   2   2   4  ...    1    1    2    2   
3    STUDENT4      1   1   1   3   1   2   1   2   1  ...    1    2    3    2   
4    STUDENT5      2   2   1   3   2   2   1   3   1  ...    2    1    2    2   
..           ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ...  ...  ...  ...   
140  STUDENT141    2   1   2   3   1   1   2   1   1  ...    1    1    2    1   
141  STUDENT142    1   1   2   4   2   2   2   1   4  ...    1    1    3    2   
142  STUDENT143    1   1   1   4   2   2   2   1   1  ...    1    1    3    3   
143  STUDENT144    2   1   2   4   1   1   1   5   2  ...    2    1    2    1   
144  STUDENT145    1   1   1   5   2   2   2   3   1  ...    2    1    3    2   

     27   28   29   30   CO

,1,2,3,4,5,6,7,8,9,10,...,23,24,25,26,27,28,29,30,COURSE ID,GRADE
count,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,...,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000
mean,1.620690,1.600000,1.944828,3.572414,1.662069,1.600000,1.579310,1.627586,1.620690,1.731034,...,1.337931,1.165517,2.544828,2.055172,2.393103,1.806897,3.124138,2.724138,4.131034,3.227586
std,0.613154,0.491596,0.537216,0.805750,0.474644,0.491596,0.495381,1.020245,1.061112,0.783999,...,0.614870,0.408483,0.564940,0.674736,0.604343,0.810492,1.301083,0.916536,3.260145,2.197678
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,1.000000,1.000000,2.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,2.000000,2.000000,2.000000,1.000000,2.000000,2.000000,1.000000,1.000000
50%,2.000000,2.000000,2.000000,3.000000,2.000000,2.000000,2.000000,1.000000,1.000000,2.000000,...,1.000000,1.000000,3.000000,2.000000,2.000000,2.000000,3.000000,3.000000,3.000000,3.000000
75%,2.000000,2.000000,2.000000,4.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,1.000000,3.000000,3.000000,3.000000,2.000000,4.000000,3.000000,7.000000,5.000000
max,3.000000,2.000000,3.000000,5.000000,2.000000,2.000000,2.000000,5.000000,4.000000,4.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,5.000000,4.000000,9.000000,7.000000
